In [ ]:
import os
os.remove('snorkel.db') #need these two files to exist in sandbox directory. Just copy from tutorials
os.system('cp snorkel.db\ corpus snorkel.db');

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession
session = SnorkelSession()

import sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

In [ ]:
from snorkel.models import Corpus

corpus = session.query(Corpus).filter(Corpus.name == 'Hardware').one()
print "%s contains %d Documents" % (corpus, len(corpus))


In [ ]:
from snorkel.models import candidate_subclass

Part = candidate_subclass('Part', ['part'])

from hardware_utils import OmniNgramsPart, OmniNgramsTemp

part_ngrams = OmniNgramsPart(n_max=3, split_tokens=None)

from hardware_utils import load_extended_parts_dict
from snorkel.matchers import DictionaryMatch, RegexMatchEach, RegexMatchSpan, Union

# filename = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
# parts_dict = load_extended_parts_dict(filename)
# print "Loaded %d part numbers." % len(parts_dict)
# parts_matcher = DictionaryMatch(d=parts_dict)

eeca_matcher = RegexMatchSpan(rgx='([b]{1}[abcdefklnpqruyz]{1}[\swxyz]?[0-9]{3,4}[\s]?[A-Z]{0,2}[0-9]?([-][A-Z0-9]{1,3})?)')
jedec_matcher = RegexMatchSpan(rgx='([123]N\d{3,4}[A-Z]?)')
jis_matcher = RegexMatchSpan(rgx='(2S[abcdefghjkmqrstvz]{1}[\d]{2,4})')
others_matcher = RegexMatchSpan(rgx='((NSVBC|SMBT|MJ|MJE|MPS|MRF|RCA|TIP|ZTX|ZT|TIS|TIPL|DTC|MMBT|PZT){1}[\d]{2,4}[A-Z]{0,3}([-][A-Z0-9]{0,3})?)')
parts_matcher = Union(eeca_matcher, jedec_matcher, jis_matcher, others_matcher)

In [ ]:
from hardware_utils import get_gold_parts, get_gold_dict

gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
gold_parts = get_gold_parts(gold_file)

In [ ]:
# Confirm that all gold parts are findable by regular expression
tp = []
fn = []
for g in gold_parts:
    if (re.match(eeca_matcher.rgx, g, flags=re.I) is not None 
        or re.match(jedec_matcher.rgx, g, flags=re.I)
        or re.match(jis_matcher.rgx, g, flags=re.I)
        or re.match(others_matcher.rgx, g, flags=re.I) is not None):
        tp.append(g)
    else:
        fn.append(g)
print len(tp), len(fn)

In [ ]:
print tp[:5]
print '---------------------'
print fn[:5]

In [ ]:
# import re
# from collections import defaultdict

# rgx = re.compile(r'\w*\d+(\-[\w]+|[A-Z]+[\w\-\/]*)')
# # rgx = re.compile(r'\w+-\w*-\w+')
# suffixes = defaultdict(int)
# print len(gold_parts)
# for g in gold_parts:
#     m = rgx.match(g)
#     if m:
#         suffixes[m.group(1)] += 1
#         if m.group(1)=='-25ZL1GA':
#             print g
# print len(suffixes)

In [ ]:
# # from collections import Counter
# counter = Counter(suffixes.values())
# print counter
# print suffixes

In [ ]:
from snorkel.candidates import CandidateExtractor

ce = CandidateExtractor(Part, [part_ngrams], [parts_matcher])

In [ ]:
%time candidates = ce.extract(corpus.documents, 'Hardware Candidates', session)
print "%s contains %d Candidates" % (candidates, len(candidates))

In [ ]:
print len(candidates)
print candidates[0]

In [ ]:
for c in candidates[:3]: print c

In [ ]:
# TEMP
from hardware_utils import entity_level_total_recall

filename = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
(tp, fp, fn) = entity_level_total_recall(candidates, filename, 'stg_temp_min', relation=False)